In [1]:
import sys
root = '/home/featurize/data'
sys.path.append(f'{root}/pikachu/utils')
sys.path.append(f'{root}/pikachu/third')
sys.path.append(f'{root}/pikachu')
sys.path.append(f'{root}/pikachu/projects/ai/naic_seg')

In [2]:
import gezi
import melt
import melt as mt
from gezi import tqdm
from gezi.metrics.image.semantic_seg import Evaluator
import lele
import husky
import pandas as pd
import numpy as np
import tensorflow as tf
from absl import app, flags
FLAGS = flags.FLAGS
from gseg import config
from gseg.config import *
from gseg.dataset import Dataset
from gseg.evaluate import get_eval_fn
from gseg import util
from gseg.util import get_infer_fn
from gseg.loss import get_loss_fn
from gseg.metrics import get_metrics
from gseg.model import get_model

In [3]:
command = 'python ./train.py --model=Model --model_dir=../working/v1/common.sh --train_input=../input/tfrecords/train/*/* --test_input=../input/tfrecords/test/*/* --model=Model --restore_exclude=global_step,ignore,learning_rate --global_epoch=0 --global_step=0 --learning_rate=0.01 --min_learning_rate=1e-06 --optimizer=bert-adam --batch_size=32 --interval_steps=100 --valid_interval_steps=100 --write_summary --write_metric_summary --freeze_graph_final=0 --vie=0.25 --allow_cpu=0 --async_valid --model=Model --mname=baseline --mn=baseline.10epoch --mode=valid --tb_image=0'
GCS_ROOT = '/home/featurize/data'
GCS_PATH = f'{GCS_ROOT}/tfrecords'
argv = command.strip().split()
FLAGS(argv)
FLAGS.async_valid = False
FLAGS.allow_cpu = False
FLAGS.train_input = f'{GCS_PATH}/train/*/*'
FLAGS.test_input=f'{GCS_PATH}/test/*/*'
# FLAGS.test_input=''
FLAGS.do_test = False
FLAGS.batch_parse = False
FLAGS.static_input = True
FLAGS.buffer_size = 200
FLAGS.verbose = True
FLAGS.mode = 'train'
FLAGS.fold = 0
FLAGS.tb_image = True
FLAGS.bfloat16 = False
FLAGS.write_valid_results = False
FLAGS.write_test_results = False
FLAGS.tb_image_interval = 1000000
FLAGS.learning_rate=0.01 
FLAGS.min_learning_rate=1e-06 
FLAGS.optimizer='bert-adam'

In [4]:
# 这里必须fix batch size
FLAGS.drop_remainder = True 
FLAGS.print_depth = 1
# since dataset is too big not cache 否则会socket close
FLAGS.cache_valid = False
FLAGS.epochs = 5
FLAGS.vie = 1 
FLAGS.fast_eval = False
FLAGS.mode = 'train'
FLAGS.dataset_device = 'cpu' 
FLAGS.aug_train_image = False
# FLAGS.dry_run = True
FLAGS.dry_run = False

In [5]:
def fit(dry_run=False):
  with strategy.scope():
    mt.fit(model, 
          #  loss_fn=get_loss_fn(),
           loss_fn=model.get_loss(),
           Dataset=Dataset,
           metrics=get_metrics(),
           eval_fn=get_eval_fn(),
           inference_fn=get_infer_fn(),
           dry_run=dry_run
          )

In [6]:
FLAGS.data_version = 2
FLAGS.fold = 1
ver = 9#@param
tag = f'v{ver}' if FLAGS.data_version == 2 else f'v{ver}.dv1'
FLAGS.model_dir = f'../working/{tag}/base' if FLAGS.fold == 1 else f'../working/{tag}.fold{FLAGS.fold}/base'
FLAGS.clear_first = False #@param
FLAGS.check_exists = False
FLAGS.mode = 'train'
FLAGS.custom_eval = False
FLAGS.vie = 5
FLAGS.save_interval_epochs = FLAGS.vie 
FLAGS.aug_train_image = True
FLAGS.optimizer = 'bert-adamw'
FLAGS.learning_rate = 1e-3
FLAGS.min_learning_rate = 1e-5
FLAGS.learning_rate_decay_power = 0.5
# FLAGS.lr_decay_method = 'cosine'
FLAGS.epochs =  200#@param
# FLAGS.swa_start_epoch = 50
# FLAGS.swa_lr_ratio = 0.1
# FLAGS.swa_freq = 2
FLAGS.hflip_rate = 0.5 #@param
FLAGS.vflip_rate = 0.5 #@param
FLAGS.rotate_rate =  0.5 #@param
FLAGS.backbone_weights = 'noisy-student' 
# FLAGS.resize_aug_rate = 0.2
FLAGS.dropout = 0.3
FLAGS.label_smoothing = 0.05
FLAGS.inter_activation = 'swish'
FLAGS.deeplab_sepconv = False
# FLAGS.fpn_filters = 256 #@param
# FLAGS.seg_weights = '../working/v8.dv1/sm.unet.efficientnetb4.nosiy.100epoch.1030'
# FLAGS.pretrain = 'sm.unet.efficientnetb4.nosiy.50epoch.adamw.power05.1030'
# FLAGS.start_epoch = 50
# FLAGS.teacher = '../working/v8/ensemble.gpu'
# FLAGS.mix_dataset = True
# FLAGS.input = f'{GCS_ROOT}/quarter/tfrecords/train/*/*,{GCS_PATH}/train/*/*'
# FLAGS.valid_exclude = f'{GCS_PATH}'
# FLAGS.multi_scale = True #@param
# FLAGS.multi_scale_attn = True #@param
# FLAGS.multi_scale_attn_dynamic = True #@param
# FLAGS.image_size2 = [288, 288]
# FLAGS.loss_fn = 'tanimoto' #@param
# FLAGS.ce_loss_rate = 1. #@param
FLAGS.augment_level =  3#@param
# FLAGS.aug_train_image = False
FLAGS.debug = False
FLAGS.mode = 'valid'
FLAGS.load_weights_only = True
FLAGS.batch_size = 32

# FLAGS.model = 'sm.Unet' #@param
# FLAGS.backbone = 'EfficientNetB4' #@param
# FLAGS.image_size = [288, 288]
# FLAGS.mrate = 0
# # FLAGS.fpn_filters = 128
# FLAGS.mn = f'sm.Unet.EfficientNetB4.augl4.size288.mrate0.200epoch.1115' #@param
# FLAGS.pretrain = f'/home/featurize/work/naicseg/v9/{FLAGS.mn}'

FLAGS.model = 'sm.Unet' #@param
FLAGS.backbone = 'EfficientNetB4' #@param
FLAGS.image_size = [288, 288]
FLAGS.mrate = 1
# FLAGS.fpn_filters = 128
FLAGS.mn = f'sm.Unet.EfficientNetB4.augl4.size288.mrate1.200epoch.1115.swa10_aav' #@param
FLAGS.pretrain = f'/home/featurize/work/naicseg/v9/{FLAGS.mn}'

# FLAGS.model = 'sm.FPN' #@param
# FLAGS.backbone = 'EfficientNetB4' #@param
# # FLAGS.image_size = [288, 288]
# FLAGS.mrate = 1
# # FLAGS.fpn_filters = 128
# FLAGS.mn = 'sm.FPN.EfficientNetB4.augl4.size256.mrate1.200epoch.1115.swa10_aav' #@param
# FLAGS.pretrain = f'/home/featurize/work/naicseg/v9/{FLAGS.mn}'

# FLAGS.model = 'bread.DeeplabV3Plus' #@param
# FLAGS.backbone = 'EfficientNetB6' #@param
# # FLAGS.image_size = [288, 288]
# FLAGS.mrate = 1
# # FLAGS.fpn_filters = 128
# FLAGS.mn = 'bread.DeeplabV3Plus.EfficientNetB6.augl4.size256.mrate1.200epoch.1115.swa10_aav' #@param
# FLAGS.pretrain = f'/home/featurize/work/naicseg/v9/{FLAGS.mn}'

# FLAGS.model = 'sm.FPN' #@param
# FLAGS.backbone = 'EfficientNetB6' #@param
# # FLAGS.image_size = [288, 288]
# # FLAGS.mrate = True
# # FLAGS.fpn_filters = 128
# mn = 'sm.FPN.EfficientNetB6.augl3.200epoch'
# FLAGS.pretrain = f'/home/featurize/work/naicseg/v9/{mn}'

print(FLAGS.pretrain)

/home/featurize/work/naicseg/v9/sm.Unet.EfficientNetB4.augl4.size288.mrate1.200epoch.1115.swa10_aav


In [7]:
config.init()
mt.init()
strategy = mt.distributed.get_strategy()
with strategy.scope():
  model = get_model(FLAGS.model)

2020-11-17 16:54:25 0:00:00 fcntl.floc with lock_file /home/featurize/.melt.lock (If hang here means other programs calling melt.init have not finished yet)
2020-11-17 16:54:25 0:00:00 Tf dataset and Tf model train in Eager mode, keras True, distributed:False
2020-11-17 16:54:25 0:00:00 log_level: 20 (try --debug to show more or --log_level=(> 20) to show less(no INFO), try --verbose to show train/valid loss intervaly)
2020-11-17 16:54:25 0:00:00 batch_size: 32 eval_batch_size: 32 batch_size_per_gpu: 32 num_gpus: 1 gpu: [0] CUDA_VISIABLE_DEVICES=[] work_mode: valid distributed: False horovod: False
2020-11-17 16:54:26 0:00:00 model: [sm.Unet.EfficientNetB4.augl4.size288.mrate1.200epoch.1115.swa10_aav] model_dir: [../working/v9/sm.Unet.EfficientNetB4.augl4.size288.mrate1.200epoch.1115.swa10_aav]
2020-11-17 16:54:26 0:00:00 ori_image_size: [256, 256] image_size: [288, 288]


Segmentation Models: using `tf.keras` framework.
Segmentation Models: using `tf.keras` framework.


2020-11-17 16:54:30 0:00:04 model: <tensorflow.python.keras.engine.functional.Functional object at 0x7ff70075cfd0> backbone: EfficientNetB4
2020-11-17 16:54:30 0:00:04 using preprocess of sm for EfficientNetB4 efficientnetb4
2020-11-17 16:54:30 0:00:04 preprocess: <function preprocess_input at 0x7ff7006a2320>
2020-11-17 16:54:30 0:00:04 use_tta: False tta: ['flip_left_right', 'flip_up_down'] []


In [8]:
FLAGS.custom_eval = False
FLAGS.tta = False
FLAGS.num_valid = 100
fit()
mt.save_model(model, f'{FLAGS.model_dir}/model.h5')

2020-11-17 16:54:30 0:00:04 Round: 0 mode: valid train_input:[None] valid_input:[*] train_dirs:[0] valid_dir: /home/featurize/data/quarter/tfrecords/train/*/* do_valid: True do_test: False


2020-11-17 16:54:34 0:00:08 num_train_examples: 90000 num_steps_per_epoch: 2813
2020-11-17 16:54:34 0:00:08 num_valid_examples: 100 num_valid_steps_per_epoch: 4
2020-11-17 16:54:34 0:00:08 round: 0 loss_fn: <function Model.loss_wrapper.<locals>.loss_fn at 0x7ff7006a2b00>
2020-11-17 16:54:34 0:00:08 optimizer: <husky.optimization.AdamWeightDecay object at 0x7ff88835b650> lr: <husky.optimization.WarmUp object at 0x7ff700287090> init_lr: 0.001
2020-11-17 16:54:34 0:00:08 total_steps: 562600 warmup_steps: 56260 end_lr: 1e-05
2020-11-17 16:54:35 0:00:09 load_weights: [/home/featurize/work/naicseg/v9/sm.Unet.EfficientNetB4.augl4.size288.mrate1.200epoch.1115.swa10_aav/model.h5]


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sm-unet-efficientnetb4 (Func [(None, 288, 288, 15), (N 23092895  
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  26895     
_________________________________________________________________
efficientnet-b4 (Functional) (None, 9, 9, 1792)        17673816  
_________________________________________________________________
sm-unet-efficientnetb4_notop (None, 288, 288, 8)       23091800  
Total params: 23,119,790
Trainable params: 22,993,598
Non-trainable params: 126,192
_________________________________________________________________
4/4 [==============================] - 1s 139ms/step - loss: 0.5964 - FWIoU: 0.8086 - MIoU: 0.8194 - ACC/pixe

2020-11-17 16:55:18 0:00:52 water                |true:0.0290 pred:0.0270 acc:0.9365 recall:0.8713 iou:0.8226 fwiou:0.9229|
2020-11-17 16:55:18 0:00:52 track_road           |true:0.0760 pred:0.0736 acc:0.9146 recall:0.8852 iou:0.8176 fwiou:0.7920|
2020-11-17 16:55:18 0:00:52 build                |true:0.1271 pred:0.1298 acc:0.9259 recall:0.9453 iou:0.8788 fwiou:0.7689|
2020-11-17 16:55:18 0:00:52 track_airport        |true:0.0000 pred:0.0000 acc:nan recall:nan iou:nan fwiou:nan|
2020-11-17 16:55:18 0:00:52 other_park           |true:0.0809 pred:0.0809 acc:0.9434 recall:0.9438 iou:0.8932 fwiou:0.8704|
2020-11-17 16:55:18 0:00:52 other_playground     |true:0.0090 pred:0.0093 acc:0.9585 recall:0.9884 iou:0.9478 fwiou:0.9929|
2020-11-17 16:55:18 0:00:52 arable_natural       |true:0.2111 pred:0.2161 acc:0.9078 recall:0.9292 iou:0.8490 fwiou:0.5221|
2020-11-17 16:55:18 0:00:52 arable_greenhouse    |true:0.0143 pred:0.0148 acc:0.8845 recall:0.9184 iou:0.8201 fwiou:0.9615|
2020-11-17 16:55:18 

In [9]:
FLAGS.custom_eval = False
FLAGS.tta = True
FLAGS.tta_fns = ['flip_left_right']
FLAGS.tta_use_original = False
FLAGS.num_valid = 100
fit()
mt.save_model(model, f'{FLAGS.model_dir}/model_lr.h5')

2020-11-17 16:55:20 0:00:54 Round: 1 mode: valid train_input:[None] valid_input:[*] train_dirs:[0] valid_dir: /home/featurize/data/quarter/tfrecords/train/*/* do_valid: True do_test: False


2020-11-17 16:55:20 0:00:55 num_train_examples: 90000 num_steps_per_epoch: 2813
2020-11-17 16:55:20 0:00:55 num_valid_examples: 100 num_valid_steps_per_epoch: 4
2020-11-17 16:55:20 0:00:55 round: 1 loss_fn: <function Model.loss_wrapper.<locals>.loss_fn at 0x7ff8883453b0>
2020-11-17 16:55:20 0:00:55 optimizer: <husky.optimization.AdamWeightDecay object at 0x7ff88835b650> lr: 0.001 init_lr: 0.001
2020-11-17 16:55:20 0:00:55 total_steps: 562600 warmup_steps: 56260 end_lr: 1e-05


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sm-unet-efficientnetb4 (Func [(None, 288, 288, 15), (N 23092895  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1792)              0         
_________________________________________________________________
dense (Dense)                (None, 15)                26895     
_________________________________________________________________
efficientnet-b4 (Functional) (None, 9, 9, 1792)        17673816  
_________________________________________________________________
sm-unet-efficientnetb4_notop (None, 288, 288, 8)       23091800  
Total params: 23,119,790
Trainable params: 22,993,598
Non-trainable params: 126,192
_________________________________________________________________
4/4 [==============================] - 1s 138ms/step - loss: 0.5958 - FWIoU: 0.8090 - MIoU: 0.8186 - ACC/pixe

2020-11-17 16:55:21 0:00:56 water                |true:0.0290 pred:0.0277 acc:0.9171 recall:0.8758 iou:0.8115 fwiou:0.9181|
2020-11-17 16:55:21 0:00:56 track_road           |true:0.0760 pred:0.0734 acc:0.9162 recall:0.8849 iou:0.8186 fwiou:0.7931|
2020-11-17 16:55:21 0:00:56 build                |true:0.1271 pred:0.1297 acc:0.9274 recall:0.9460 iou:0.8807 fwiou:0.7726|
2020-11-17 16:55:21 0:00:56 track_airport        |true:0.0000 pred:0.0000 acc:nan recall:nan iou:nan fwiou:nan|
2020-11-17 16:55:21 0:00:56 other_park           |true:0.0809 pred:0.0806 acc:0.9447 recall:0.9417 iou:0.8925 fwiou:0.8696|
2020-11-17 16:55:21 0:00:56 other_playground     |true:0.0090 pred:0.0094 acc:0.9507 recall:0.9889 iou:0.9407 fwiou:0.9919|
2020-11-17 16:55:21 0:00:56 arable_natural       |true:0.2111 pred:0.2157 acc:0.9079 recall:0.9278 iou:0.8480 fwiou:0.5187|
2020-11-17 16:55:21 0:00:56 arable_greenhouse    |true:0.0143 pred:0.0148 acc:0.8859 recall:0.9164 iou:0.8197 fwiou:0.9614|
2020-11-17 16:55:21 

In [10]:
FLAGS.custom_eval = False
FLAGS.tta = True
FLAGS.tta_fns = ['flip_up_down']
FLAGS.tta_use_original = False
FLAGS.num_valid = 100
fit()
mt.save_model(model, f'{FLAGS.model_dir}/model_ud.h5')

2020-11-17 16:55:23 0:00:57 Round: 2 mode: valid train_input:[None] valid_input:[*] train_dirs:[0] valid_dir: /home/featurize/data/quarter/tfrecords/train/*/* do_valid: True do_test: False


2020-11-17 16:55:24 0:00:58 num_train_examples: 90000 num_steps_per_epoch: 2813
2020-11-17 16:55:24 0:00:58 num_valid_examples: 100 num_valid_steps_per_epoch: 4
2020-11-17 16:55:24 0:00:58 round: 2 loss_fn: <function Model.loss_wrapper.<locals>.loss_fn at 0x7ff7002fbef0>
2020-11-17 16:55:24 0:00:58 optimizer: <husky.optimization.AdamWeightDecay object at 0x7ff88835b650> lr: 0.001 init_lr: 0.001
2020-11-17 16:55:24 0:00:58 total_steps: 562600 warmup_steps: 56260 end_lr: 1e-05


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sm-unet-efficientnetb4 (Func [(None, 288, 288, 15), (N 23092895  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1792)              0         
_________________________________________________________________
dense (Dense)                (None, 15)                26895     
_________________________________________________________________
efficientnet-b4 (Functional) (None, 9, 9, 1792)        17673816  
_________________________________________________________________
sm-unet-efficientnetb4_notop (None, 288, 288, 8)       23091800  
Total params: 23,119,790
Trainable params: 22,993,598
Non-trainable params: 126,192
_________________________________________________________________
4/4 [==============================] - 1s 127ms/step - loss: 0.5966 - FWIoU: 0.8066 - MIoU: 0.8158 - ACC/pixe

2020-11-17 16:55:25 0:00:59 water                |true:0.0290 pred:0.0273 acc:0.9260 recall:0.8710 iou:0.8143 fwiou:0.9193|
2020-11-17 16:55:25 0:00:59 track_road           |true:0.0760 pred:0.0736 acc:0.9129 recall:0.8842 iou:0.8155 fwiou:0.7895|
2020-11-17 16:55:25 0:00:59 build                |true:0.1271 pred:0.1298 acc:0.9250 recall:0.9447 iou:0.8775 fwiou:0.7665|
2020-11-17 16:55:25 0:00:59 track_airport        |true:0.0000 pred:0.0000 acc:nan recall:nan iou:nan fwiou:nan|
2020-11-17 16:55:25 0:00:59 other_park           |true:0.0809 pred:0.0808 acc:0.9426 recall:0.9414 iou:0.8903 fwiou:0.8669|
2020-11-17 16:55:25 0:00:59 other_playground     |true:0.0090 pred:0.0093 acc:0.9584 recall:0.9841 iou:0.9437 fwiou:0.9924|
2020-11-17 16:55:25 0:00:59 arable_natural       |true:0.2111 pred:0.2140 acc:0.9105 recall:0.9232 iou:0.8464 fwiou:0.5137|
2020-11-17 16:55:25 0:00:59 arable_greenhouse    |true:0.0143 pred:0.0148 acc:0.8861 recall:0.9180 iou:0.8211 fwiou:0.9617|
2020-11-17 16:55:25 

In [13]:
# FLAGS.custom_eval = False
# FLAGS.tta = True
# FLAGS.tta_fns = ['flip_left_right']
# FLAGS.tta_use_original = True
# fit()
# mt.save_model(model, f'{FLAGS.model_dir}/model_tta_lr.h5')

In [12]:
# FLAGS.custom_eval = False
# FLAGS.tta = True
# FLAGS.tta_fns = ['flip_up_down']
# FLAGS.tta_use_original = True
# fit()
# mt.save_model(model, f'{FLAGS.model_dir}/model_tta_ud.h5')

In [11]:
# FLAGS.custom_eval = False
# FLAGS.tta = True
# FLAGS.tta_fns = ['flip_left_right', 'flip_up_down']
# FLAGS.tta_use_original = True
# fit()
# mt.save_model(model, f'{FLAGS.model_dir}/model_tta.h5')

In [11]:
def _predict(model, imgs, tta=False):
  if not tta:
    pred = model.predict(imgs)
  else:
    imgs_list = [imgs, tf.image.flip_left_right(imgs), tf.image.flip_up_down(imgs)]
    imgs = tf.concat(imgs_list, axis=0)
    res = np.split(model.predict(imgs), len(imgs_list))
    res[1] = tf.image.flip_left_right(res[1]).numpy()
    res[2] = tf.image.flip_up_down(res[2]).numpy()
    # pred = np.mean(res, axis=0)
    pred = (res[0] + res[1] + res[2]) / 3.
  return pred
    
def predicts(model, indir='../input/eval.naic/image', odir='../input/out', batch_size=4, tta=False, num_imgs=0):
  gezi.try_mkdir(odir)
  os.system(f'rm -rf {odir}/*')
  img_paths = glob.glob(f'{indir}/*')
  num_imgs = num_imgs or len(img_paths)
  for i in tqdm(range(0, num_imgs, batch_size), desc='predict', ascii=True, leave=True):
    begin = i
    end = min(i + batch_size, num_imgs)
    imgs = np.asarray([gezi.imread(img_path) for img_path in img_paths[begin:end]])
    logits = _predict(model, imgs, tta)
    preds = logits.argmax(-1)
    for img_path, pred in zip(img_paths[begin:end], preds):
      index = os.path.basename(img_path).split('.')[0]
      cv2.imwrite(os.path.join(odir, f'{index}.png'), pred)
      
CLASSES = ['water', 'track_road', 'build', 'track_airport', 'other_park', 'other_playground', 'arable_natural', 'arable_greenhouse',
           'grass_natural', 'grass_greenbelt', 'forest_natural', 'forest_planted', 'bare_natural', 'bare_planted', 'other_other']
NUM_CLASSES = len(CLASSES)
def predicts_eval(model, indir='../input/eval.naic/image', odir='../input/out', batch_size=4, tta=False, num_imgs=0):
  m = {}
  for i in range(17):
    if i < 4:
      m[i + 1] = i
    else:
      m[i + 1] = i - 2

  gezi.try_mkdir(odir)
  os.system(f'rm -rf {odir}/*')
  img_paths = glob.glob(f'{indir}/*')
  label_dir = indir.replace('image', 'label')
  num_imgs = num_imgs or len(img_paths)
  evaluator = Evaluator(CLASSES)
  key_metric = 'FWIoU'
  class_mious = 0.
  t = tqdm(range(0, num_imgs, batch_size), desc='predict', ascii=True, leave=True)
  for i in t:
    begin = i
    end = min(i + batch_size, num_imgs)
    imgs = np.asarray([gezi.imread(img_path) for img_path in img_paths[begin:end]])
    logits = _predict(model, imgs, tta)
    preds = logits.argmax(-1)
    for img_path, pred in zip(img_paths[begin:end], preds):
      index = os.path.basename(img_path).split('.')[0]
      cv2.imwrite(os.path.join(odir, f'{index}.png'), pred)
      
      label_file = f'{label_dir}/{index}.png'
      label = cv2.imread(label_file, cv2.IMREAD_UNCHANGED).astype(np.uint8)
      f = np.vectorize(lambda x: m[x])
      label = f(label).astype(np.uint8)
      evaluator.eval_each(label[np.newaxis,:, :], pred[np.newaxis,:, :], metric=key_metric)
      binary_label = np.bincount(label.reshape(-1), minlength=NUM_CLASSES).astype(np.bool)
      binary_pred = np.bincount(pred.reshape(-1), minlength=NUM_CLASSES).astype(np.bool)
      intersections = binary_label * binary_pred
      intersection = np.sum(intersections, axis=-1)
      union = np.sum(binary_label, axis=-1) + np.sum(binary_pred, axis=-1) - intersection
      class_miou = np.mean(intersection / union)
      class_mious += class_miou
      t.set_postfix({key_metric: evaluator.eval_once(key_metric), 'IMAGE/CLASS/MIoU': class_mious / (i + 1)})

  res = evaluator.eval_once()
  res['IMAGE/CLASS/MIoU'] = class_mious / i
  gezi.set('evaluator', evaluator)
  return res

In [12]:
mt.init_flags()
FLAGS = mt.get_flags()
FLAGS.batch_parse = False
batch_size = 32
mt.set_global('batch_size', batch_size) # loss fn used / mt.batch_size()
# eval_files = gezi.list_files('../input/quarter/tfrecords/train/1/*')
eval_files = gezi.list_files('../input/quarter/tfrecords/train/1/21.*')
eval_dataset = Dataset('valid').make_batch(batch_size, eval_files)
steps = -(-mt.get_num_records(eval_files) // batch_size)
FLAGS.NUM_CLASSES = NUM_CLASSES

In [13]:
def eval(model):
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = get_metrics()
  model.compile('sgd', loss_fn, metrics=metrics)
  res = model.evaluate(eval_dataset, steps=steps, return_dict=True)
  gezi.pprint_dict(res)
  cm = mt.distributed.sum_merge(metrics[0].get_cm())
  infos = util.get_infos_from_cm(cm, CLASSES)
  res.update(infos)
  return res

In [14]:
model_paths = [
  '../working/v9/sm.Unet.EfficientNetB4.augl4.size288.mrate1.200epoch.1115.swa10_asv/model.h5',
  '../working/v9/sm.FPN.EfficientNetB4.augl4.size256.mrate1.200epoch.1115.swa10_asv/model.h5',
  '../working/v9/bread.DeeplabV3Plus.EfficientNetB6.augl4.size256.mrate1.200epoch.1115.swa10_asv/model.h5'
]
models = []
for model_path in tqdm(model_paths):
  model = mt.load_model(model_path)
  models.append(model)
model = mt.EnsembleModel(models).get_model()
# model.summary()
res = eval(model)


104/105 [============================>.] - 62s 596ms/step - loss: 0.4045 - FWIoU: 0.7829 - MIoU: 0.7183 - ACC/pixel: 0.8761 - ACC/class: 0.8017

 +---------+----------+----------+-------------+-------------+
|    loss |    FWIoU |     MIoU |   ACC/pixel |   ACC/class |
|---------+----------+----------+-------------+-------------|
| 0.40452 | 0.782869 | 0.718277 |    0.876074 |    0.801702 |
+---------+----------+----------+-------------+-------------+
water                |true:0.0257 pred:0.0252 acc:0.8930 recall:0.8748 iou:0.7919 fwiou:0.9198|
track_road           |true:0.0776 pred:0.0764 acc:0.9019 recall:0.8884 iou:0.8102 fwiou:0.7790|
build                |true:0.1217 pred:0.1252 acc:0.9180 recall:0.9440 iou:0.8706 fwiou:0.7637|
track_airport        |true:0.0002 pred:0.0000 acc:nan recall:0.0000 iou:0.0000 fwiou:0.9973|
other_park           |true:0.0654 pred:0.0653 acc:0.9315 recall:0.9300 iou:0.8705 fwiou:0.8730|
other_playground     |true:0.0036 pred:0.0032 acc:0.9621 recall:0.

In [15]:
model_paths = [
  '../working/v9/sm.Unet.EfficientNetB4.augl4.size288.mrate1.200epoch.1115.swa10_asv/model.h5',
  '../working/v9/sm.FPN.EfficientNetB4.augl4.size256.mrate1.200epoch.1115.swa10_asv/model_lr.h5',
  '../working/v9/bread.DeeplabV3Plus.EfficientNetB6.augl4.size256.mrate1.200epoch.1115.swa10_asv/model_ud.h5'
]
models = []
for model_path in tqdm(model_paths):
  model = mt.load_model(model_path)
  models.append(model)
model = mt.EnsembleModel(models).get_model()
# model.summary()
res = eval(model)


104/105 [============================>.] - 62s 597ms/step - loss: 0.4042 - FWIoU: 0.7828 - MIoU: 0.7185 - ACC/pixel: 0.8760 - ACC/class: 0.8018

 +----------+----------+----------+-------------+-------------+
|     loss |    FWIoU |     MIoU |   ACC/pixel |   ACC/class |
|----------+----------+----------+-------------+-------------|
| 0.404174 | 0.782847 | 0.718485 |    0.876039 |    0.801849 |
+----------+----------+----------+-------------+-------------+
water                |true:0.0257 pred:0.0252 acc:0.8933 recall:0.8752 iou:0.7923 fwiou:0.9200|
track_road           |true:0.0776 pred:0.0764 acc:0.9019 recall:0.8876 iou:0.8094 fwiou:0.7781|
build                |true:0.1217 pred:0.1252 acc:0.9182 recall:0.9442 iou:0.8709 fwiou:0.7643|
track_airport        |true:0.0002 pred:0.0000 acc:nan recall:0.0000 iou:0.0000 fwiou:0.9973|
other_park           |true:0.0654 pred:0.0653 acc:0.9311 recall:0.9308 iou:0.8709 fwiou:0.8734|
other_playground     |true:0.0036 pred:0.0032 acc:0.9586 reca

In [14]:
model_paths = [
  '../working/v9/sm.Unet.EfficientNetB4.augl4.size288.mrate1.200epoch.1115.swa10_aav/model.h5',
  '../working/v9/sm.FPN.EfficientNetB4.augl4.size256.mrate1.200epoch.1115.swa10_aav/model_lr.h5',
  '../working/v9/bread.DeeplabV3Plus.EfficientNetB6.augl4.size256.mrate1.200epoch.1115.swa10_aav/model_ud.h5'
]
models = []
for model_path in tqdm(model_paths):
  model = mt.load_model(model_path)
  models.append(model)
model = mt.EnsembleModel(models).get_model()
# model.summary()
res = eval(model)


104/105 [============================>.] - 62s 598ms/step - loss: 0.3198 - FWIoU: 0.8204 - MIoU: 0.7582 - ACC/pixel: 0.9001 - ACC/class: 0.8287

 +----------+----------+----------+-------------+-------------+
|     loss |    FWIoU |     MIoU |   ACC/pixel |   ACC/class |
|----------+----------+----------+-------------+-------------|
| 0.319758 | 0.820436 | 0.758236 |    0.900063 |    0.828738 |
+----------+----------+----------+-------------+-------------+
water                |true:0.0257 pred:0.0252 acc:0.9126 recall:0.8973 iou:0.8263 fwiou:0.9331|
track_road           |true:0.0776 pred:0.0764 acc:0.9159 recall:0.9013 iou:0.8324 fwiou:0.8049|
build                |true:0.1217 pred:0.1254 acc:0.9218 recall:0.9493 iou:0.8785 fwiou:0.7782|
track_airport        |true:0.0002 pred:0.0000 acc:nan recall:0.0000 iou:0.0000 fwiou:0.9973|
other_park           |true:0.0654 pred:0.0653 acc:0.9507 recall:0.9499 iou:0.9053 fwiou:0.9072|
other_playground     |true:0.0036 pred:0.0033 acc:0.9645 reca

In [15]:
mt.save_model(model, '../working/ensemble/model_1117.h5')

In [16]:
model_paths = [
  '../working/v9/sm.Unet.EfficientNetB4.augl4.size288.mrate1.200epoch.1115/model.h5',
  '../working/v9/sm.FPN.EfficientNetB4.augl4.size256.mrate1.200epoch.1115/model_lr.h5',
  '../working/v9/bread.DeeplabV3Plus.EfficientNetB6.augl4.size256.mrate1.200epoch.1115/model_ud.h5'
]
models = []
for model_path in tqdm(model_paths):
  model = mt.load_model(model_path)
  models.append(model)
model = mt.EnsembleModel(models).get_model()
# model.summary()
res = eval(model)


104/105 [============================>.] - 62s 599ms/step - loss: 0.4128 - FWIoU: 0.7811 - MIoU: 0.7135 - ACC/pixel: 0.8749 - ACC/class: 0.7977

 +----------+----------+----------+-------------+-------------+
|     loss |    FWIoU |     MIoU |   ACC/pixel |   ACC/class |
|----------+----------+----------+-------------+-------------|
| 0.412825 | 0.781128 | 0.713539 |    0.874905 |    0.797687 |
+----------+----------+----------+-------------+-------------+
water                |true:0.0257 pred:0.0256 acc:0.8808 recall:0.8773 iou:0.7842 fwiou:0.9169|
track_road           |true:0.0776 pred:0.0764 acc:0.9016 recall:0.8872 iou:0.8088 fwiou:0.7775|
build                |true:0.1217 pred:0.1251 acc:0.9185 recall:0.9437 iou:0.8708 fwiou:0.7641|
track_airport        |true:0.0002 pred:0.0000 acc:nan recall:0.0000 iou:0.0000 fwiou:0.9973|
other_park           |true:0.0654 pred:0.0654 acc:0.9298 recall:0.9303 iou:0.8693 fwiou:0.8718|
other_playground     |true:0.0036 pred:0.0031 acc:0.9529 reca

In [16]:
model_paths = [
  '../working/v9/sm.Unet.EfficientNetB4.augl4.size288.mrate1.200epoch.1115/model.h5',
  '../working/v9/sm.FPN.EfficientNetB4.augl4.size256.mrate1.200epoch.1115/model.h5',
  '../working/v9/bread.DeeplabV3Plus.EfficientNetB6.augl4.size256.mrate1.200epoch.1115/model.h5'
]
models = []
for model_path in tqdm(model_paths):
  model = mt.load_model(model_path)
  models.append(model)
model = mt.EnsembleModel(models).get_model()
# model.summary()
res = eval(model)


104/105 [============================>.] - 62s 596ms/step - loss: 0.4132 - FWIoU: 0.7811 - MIoU: 0.7142 - ACC/pixel: 0.8749 - ACC/class: 0.7983

 +----------+----------+---------+-------------+-------------+
|     loss |    FWIoU |    MIoU |   ACC/pixel |   ACC/class |
|----------+----------+---------+-------------+-------------|
| 0.413181 | 0.781113 | 0.71421 |    0.874936 |    0.798327 |
+----------+----------+---------+-------------+-------------+
water                |true:0.0257 pred:0.0256 acc:0.8810 recall:0.8770 iou:0.7841 fwiou:0.9168|
track_road           |true:0.0776 pred:0.0765 acc:0.8999 recall:0.8874 iou:0.8077 fwiou:0.7761|
build                |true:0.1217 pred:0.1251 acc:0.9182 recall:0.9434 iou:0.8703 fwiou:0.7631|
track_airport        |true:0.0002 pred:0.0000 acc:nan recall:0.0000 iou:0.0000 fwiou:0.9973|
other_park           |true:0.0654 pred:0.0653 acc:0.9293 recall:0.9283 iou:0.8670 fwiou:0.8696|
other_playground     |true:0.0036 pred:0.0031 acc:0.9589 recall:0.

In [17]:
model_paths = [
  '../working/v9/sm.Unet.EfficientNetB4.augl4.size288.mrate1.200epoch.1115/model.h5',
  '../working/v9/sm.FPN.EfficientNetB4.augl4.size256.mrate1.200epoch.1115/model.h5',
  '../working/v9/bread.DeeplabV3Plus.EfficientNetB6.augl4.size256.mrate1.200epoch.1115/model.h5'
]
models = []
for model_path in tqdm(model_paths):
  model = mt.load_model(model_path)
  models.append(model)
model = mt.EnsembleModel(models, weights=[1., 0.8, 0.5]).get_model()
# model.summary()
res = eval(model)


104/105 [============================>.] - 64s 615ms/step - loss: 0.6300 - FWIoU: 0.7799 - MIoU: 0.7133 - ACC/pixel: 0.8741 - ACC/class: 0.7983

 +----------+----------+----------+-------------+-------------+
|     loss |    FWIoU |     MIoU |   ACC/pixel |   ACC/class |
|----------+----------+----------+-------------+-------------|
| 0.630009 | 0.779875 | 0.713327 |    0.874141 |    0.798284 |
+----------+----------+----------+-------------+-------------+
water                |true:0.0257 pred:0.0257 acc:0.8762 recall:0.8766 iou:0.7800 fwiou:0.9153|
track_road           |true:0.0776 pred:0.0767 acc:0.8987 recall:0.8877 iou:0.8070 fwiou:0.7753|
build                |true:0.1217 pred:0.1250 acc:0.9182 recall:0.9429 iou:0.8698 fwiou:0.7623|
track_airport        |true:0.0002 pred:0.0000 acc:nan recall:0.0000 iou:0.0000 fwiou:0.9973|
other_park           |true:0.0654 pred:0.0653 acc:0.9285 recall:0.9274 iou:0.8655 fwiou:0.8682|
other_playground     |true:0.0036 pred:0.0031 acc:0.9553 reca

In [14]:
model_paths = [
  '../working/v9/sm.Unet.EfficientNetB4.augl4.size288.mrate1.200epoch.1115/model_ud.h5',
  '../working/v9/sm.FPN.EfficientNetB4.augl4.size256.mrate1.200epoch.1115/model_lr.h5',
  '../working/v9/bread.DeeplabV3Plus.EfficientNetB6.augl4.size256.mrate1.200epoch.1115/model.h5'
]
models = []
for model_path in tqdm(model_paths):
  model = mt.load_model(model_path)
  models.append(model)
model = mt.EnsembleModel(models).get_model()
# model.summary()
res = eval(model)


312/313 [============================>.] - 188s 604ms/step - loss: 0.4143 - FWIoU: 0.7808 - MIoU: 0.7138 - ACC/pixel: 0.8746 - ACC/class: 0.7983

 +----------+----------+----------+-------------+-------------+
|     loss |    FWIoU |     MIoU |   ACC/pixel |   ACC/class |
|----------+----------+----------+-------------+-------------|
| 0.414297 | 0.780783 | 0.713831 |    0.874644 |     0.79834 |
+----------+----------+----------+-------------+-------------+
water                |true:0.0272 pred:0.0272 acc:0.8832 recall:0.8836 iou:0.7911 fwiou:0.9147|
track_road           |true:0.0767 pred:0.0753 acc:0.9028 recall:0.8871 iou:0.8097 fwiou:0.7812|
build                |true:0.1213 pred:0.1249 acc:0.9156 recall:0.9430 iou:0.8676 fwiou:0.7591|
track_airport        |true:0.0001 pred:0.0000 acc:nan recall:0.0000 iou:0.0000 fwiou:0.9991|
other_park           |true:0.0664 pred:0.0659 acc:0.9363 recall:0.9294 iou:0.8741 fwiou:0.8747|
other_playground     |true:0.0033 pred:0.0030 acc:0.9484 rec

In [15]:
model_paths = [
  '../working/v9/sm.Unet.EfficientNetB4.augl4.size288.mrate1.200epoch.1115/model.h5',
  '../working/v9/sm.FPN.EfficientNetB4.augl4.size256.mrate1.200epoch.1115/model_lr.h5',
  '../working/v9/bread.DeeplabV3Plus.EfficientNetB6.augl4.size256.mrate1.200epoch.1115/model_ud.h5'
]
models = []
for model_path in tqdm(model_paths):
  model = mt.load_model(model_path)
  models.append(model)
model = mt.EnsembleModel(models).get_model()
# model.summary()

In [16]:
res = eval(model)

312/313 [============================>.] - 189s 606ms/step - loss: 0.4149 - FWIoU: 0.7807 - MIoU: 0.7137 - ACC/pixel: 0.8746 - ACC/class: 0.7983

 +----------+----------+----------+-------------+-------------+
|     loss |    FWIoU |     MIoU |   ACC/pixel |   ACC/class |
|----------+----------+----------+-------------+-------------|
| 0.414911 | 0.780725 | 0.713739 |    0.874611 |    0.798308 |
+----------+----------+----------+-------------+-------------+
water                |true:0.0272 pred:0.0272 acc:0.8831 recall:0.8838 iou:0.7912 fwiou:0.9147|
track_road           |true:0.0767 pred:0.0753 acc:0.9028 recall:0.8866 iou:0.8093 fwiou:0.7807|
build                |true:0.1213 pred:0.1249 acc:0.9155 recall:0.9429 iou:0.8674 fwiou:0.7588|
track_airport        |true:0.0001 pred:0.0000 acc:nan recall:0.0000 iou:0.0000 fwiou:0.9991|
other_park           |true:0.0664 pred:0.0659 acc:0.9364 recall:0.9297 iou:0.8744 fwiou:0.8750|
other_playground     |true:0.0033 pred:0.0030 acc:0.9476 reca

In [18]:
!ls ../working/v9/sm.Unet.EfficientNetB4.augl4.size288.mrate0.200epoch.1115

args.txt
command.txt
eval_step.txt
events.out.tfevents.1605539940.pengbo-featurize-penhuolong.3620596.0
events.out.tfevents.1605539943.pengbo-featurize-penhuolong.3620596.1
events.out.tfevents.1605539943.pengbo-featurize-penhuolong.3620596.2
events.out.tfevents.1605539943.pengbo-featurize-penhuolong.3620596.3
events.out.tfevents.1605539943.pengbo-featurize-penhuolong.3620596.4
events.out.tfevents.1605539943.pengbo-featurize-penhuolong.3620596.5
events.out.tfevents.1605539943.pengbo-featurize-penhuolong.3620596.6
events.out.tfevents.1605539943.pengbo-featurize-penhuolong.3620596.7
events.out.tfevents.1605600405.pengbo-featurize-penhuolong.3634342.0
events.out.tfevents.1605600405.pengbo-featurize-penhuolong.3634342.1
events.out.tfevents.1605600405.pengbo-featurize-penhuolong.3634342.2
events.out.tfevents.1605600406.pengbo-featurize-penhuolong.3634342.3
events.out.tfevents.1605600406.pengbo-featurize-penhuolong.3634342.4
events.out.tfevents.1605600406.pengbo-featurize-penhuolong.3634342.5